In [41]:
!pip install surprise

In [42]:
import io, os, json
from surprise import Dataset ,Reader, accuracy, SVD, dump
import pandas as pd
import numpy as np
from itertools import chain

In [43]:
song_meta = pd.read_pickle("/content/drive/My Drive/recommendation sys/song_meta_0802.pkl") # new_tags와 cluster_id가 들어있는 song_meta

In [44]:
# id_to_item dictionary

tags = pd.unique(list(chain.from_iterable(song_meta.new_tags)))
sid_to_cid = dict(zip(song_meta.id.map(str), song_meta.new_song_id.map(str)))

n_clusters = 1000
n_tags = 110

tag_to_tid = {tags[i]:str(n_clusters + i) for i in range(len(tags))}
tid_to_tag = {tid:tag for tag,tid in tag_to_tid.items()}
sid_to_song = song_meta.song_name.to_dict()

In [45]:
# validation set
# 속한 노래와 태그를 id로 mapping한 후 합쳐서 cluster_tags에 list형태로 저장

val = pd.read_json("/content/drive/My Drive/recommendation sys/0802/val.json")
val['cid'] = val.songs.map(lambda x:[sid_to_cid[str(i)] for i in x])
val['tid'] = val.tags.map(lambda x:[tag_to_tid[i] for i in x if i in tags])
val['clusters_tags'] = val.cid + val.tid
val

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,"[673, 673, 55, 833, 776, 424, 472, 666, 234, 7...",[],"[673, 673, 55, 833, 776, 424, 472, 666, 234, 7..."
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[],[],[]
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,"[166, 722, 285, 285, 290, 937, 910, 220, 726, ...",[],"[166, 722, 285, 285, 290, 937, 910, 220, 726, ..."
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,"[655, 493, 213, 956, 145, 510, 145, 485, 857, ...",[],"[655, 493, 213, 956, 145, 510, 145, 485, 857, ..."
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,"[605, 940, 764, 764, 751, 584, 940, 584]",[],"[605, 940, 764, 764, 751, 584, 940, 584]"
...,...,...,...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000,"[804, 194, 935, 87, 655, 133, 857, 803, 523, 9...",[1036],"[804, 194, 935, 87, 655, 133, 857, 803, 523, 9..."
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000,"[377, 834, 418, 613, 295, 152, 787, 489, 286, ...",[],"[377, 834, 418, 613, 295, 152, 787, 489, 286, ..."
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000,"[857, 857, 857, 857, 857, 857, 857, 857, 857, ...",[],"[857, 857, 857, 857, 857, 857, 857, 857, 857, ..."
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000,"[531, 615, 155, 347, 70, 347, 347, 615, 618]",[],"[531, 615, 155, 347, 70, 347, 347, 615, 618]"


In [46]:
tr = pd.read_json("/content/drive/My Drive/recommendation sys/0802/train.json")
tr['cid'] = tr.songs.map(lambda x:[sid_to_cid[str(i)] for i in x])
tr['tid'] = tr.tags.map(lambda x:[tag_to_tid[str(i)] for i in x if x in tags])
tr['clusters_tags'] = tr.cid + tr.tid
tr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[509, 10, 783, 550, 346, 346, 604, 745, 357, 7...",[1019],"[509, 10, 783, 550, 346, 346, 604, 745, 357, 7..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[954, 525, 804, 834, 386, 452, 273, 0, 0, 145,...",[],"[954, 525, 804, 834, 386, 452, 273, 0, 0, 145,..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[523, 370, 935, 578, 493, 630, 322, 253, 293, ...",[],"[523, 370, 935, 578, 493, 630, 322, 253, 293, ..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[451, 668, 165, 55, 396, 833, 53, 782, 510, 13...",[],"[451, 668, 165, 55, 396, 833, 53, 782, 510, 13..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[954, 236, 90, 736, 797, 156, 278, 648, 736, 2...",[1058],"[954, 236, 90, 736, 797, 156, 278, 648, 736, 2..."
...,...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,"[989, 829, 829, 829, 738, 585, 604, 284, 284, ...",[],"[989, 829, 829, 829, 738, 585, 604, 284, 284, ..."
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,"[20, 677, 517, 242, 20, 20, 517, 517, 575, 517...",[1014],"[20, 677, 517, 242, 20, 20, 517, 517, 575, 517..."
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,"[164, 108, 482, 937, 856, 937, 937, 235, 133, ...",[],"[164, 108, 482, 937, 856, 937, 937, 235, 133, ..."
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,"[731, 197, 758, 533, 731, 282, 103, 731, 731, ...",[],"[731, 197, 758, 533, 731, 282, 103, 731, 731, ..."


In [47]:
# cluster별 빈도별로 sorting

song_freq = pd.DataFrame(pd.Series(chain.from_iterable(tr.songs)).value_counts(), columns = ["freq"])
song_freq['cid'] = [sid_to_cid[str(i)] for i in song_freq.index.tolist()]
song_freq_dict = song_freq.sort_values(['cid','freq'], ascending=False).groupby('cid').apply(lambda x:x.index.tolist())
song_freq_dict

cid
0      [677410, 696302, 552458, 19022, 98058, 109237,...
1      [400365, 425869, 487049, 291096, 559596, 13501...
10     [639570, 418438, 515252, 216504, 664126, 55639...
100    [6636, 422089, 499529, 295217, 150877, 220449,...
101    [98390, 92362, 212527, 300260, 4849, 118409, 1...
                             ...                        
995    [368205, 293558, 390868, 487742, 474196, 35615...
996    [294469, 595778, 628685, 11574, 144858, 692141...
997    [296599, 401892, 216879, 284390, 498317, 69062...
998    [109219, 198549, 575769, 345834, 518930, 55636...
999    [210289, 645786, 4030, 132887, 99950, 263929, ...
Length: 1000, dtype: object

##### Prediction 하는 절차에 대해 소개하고 예제를 통해 추천 결과를 확인합니다.
* ground truth에 존재하는 노래와 태그에 맞는 노래는 잘 추천해주지만 태그는 약간 맞지 않는 부분이 있다.
* tag에 대한 rating을 줄 때 대부분이 0이고 최대값이 5이기 때문에 이들 사이에서 ranking이 잘 학습되지 않은 것으로 보인다.

In [48]:
# prediction에 이용할 모델

algo = pd.read_pickle("/content/drive/My Drive/recommendation sys/0802/algo_0802.pkl")
train_obj = pd.read_pickle("/content/drive/My Drive/recommendation sys/0802/train_obj_0802.pkl")
user_matrix = algo.pu
item_matrix = algo.qi
raw_to_inner = {int(train_obj.to_raw_iid(i)):i for i in range(n_clusters+n_tags)}

In [ ]:
sorted_idx = val.tid.map(len).sort_values().index.tolist()

In [270]:
val.iloc[sorted_idx][val.iloc[sorted_idx].tags.map(lambda x: "연인" in x)].tail(50)

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
17181,"[연인, 결혼, 청혼, 결혼식]",5082,"인생에서 단한번, 프로포즈 청혼의 순간",[],5,2020-03-16 15:03:59.000,[],[1063],[1063]
87,"[금요일밤, 연인]",146681,,"[292688, 608685, 91295, 368044, 35575, 490275,...",4,2019-01-16 22:38:53.000,"[429, 647, 890, 145, 890, 964, 647, 630, 647, ...",[1063],"[429, 647, 890, 145, 890, 964, 647, 630, 647, ..."
9303,"[커플, 연인, 설레임, 솔로]",114753,,"[358506, 128830, 222444, 610973, 386114, 29570...",58,2017-02-05 13:59:39.000,"[506, 717, 216, 857, 852, 596, 346, 857, 89, 1...",[1063],"[506, 717, 216, 857, 852, 596, 346, 857, 89, 1..."
7384,"[연인, 포근한, 주방, 부담없이]",68244,,"[412579, 95485, 215829, 315691, 592826, 117367...",6,2020-02-07 22:46:46.000,"[316, 145, 852, 79, 145, 524, 145, 680, 857, 1...",[1063],"[316, 145, 852, 79, 145, 524, 145, 680, 857, 1..."
4390,"[빼빼로데이, 연인, 설레임, 고백, 여친]",104055,,"[553754, 660566, 61757, 176650, 673260, 621362...",34,2019-12-05 15:15:18.000,"[735, 668, 278, 798, 525, 782, 849, 49, 626, 8...",[1063],"[735, 668, 278, 798, 525, 782, 849, 49, 626, 8..."
7706,"[솔로, 감성힙합, 연인, 국내힙합, 제이지]",115670,,"[109190, 637303, 181515, 18117, 546794, 685565...",220,2020-02-28 16:46:30.000,"[194, 194, 964, 194, 857, 266, 279, 690, 187, ...","[1092, 1063]","[194, 194, 964, 194, 857, 266, 279, 690, 187, ..."
113,"[커플, 발라드, 연인, 달달함]",67804,,"[315173, 419696, 263863, 664140, 648538, 47931...",6,2018-01-03 10:36:56.000,"[846, 524, 57, 395, 266, 510, 680, 954, 642, 6...","[1020, 1063]","[846, 524, 57, 395, 266, 510, 680, 954, 642, 6..."
7037,"[연인, 사랑, 너와함께]",46193,,"[617854, 268506, 700976, 191667, 94083, 79270,...",0,2019-10-31 16:39:41.000,"[370, 680, 524, 273, 680, 193, 366, 523, 49, 580]","[1063, 1046]","[370, 680, 524, 273, 680, 193, 366, 523, 49, 5..."
5619,"[고급스러운, 라운지, 새벽, 연인]",145365,,"[154648, 450120, 205939, 240532, 63953, 153540...",16,2017-07-02 10:14:38.000,"[79, 302, 857, 801, 753, 31, 31, 691, 275, 582...","[1034, 1063]","[79, 302, 857, 801, 753, 31, 31, 691, 275, 582..."
9240,"[연인, 연애, 애정]",111132,,"[37298, 183078, 164521, 643008, 68675]",1,2018-02-18 18:38:20.000,"[506, 194, 370, 754, 506]","[1063, 1104]","[506, 194, 370, 754, 506, 1063, 1104]"


# Prediction example 1 : 러브, 달달, 사랑, 연인

In [ ]:
idx = 17780
target = val.iloc[idx]

# extract clusters/tags 
values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

# convert to one-hot vector
s = np.eye(n_values)[values].sum(axis=0)

# calcuate similarity
sim = SVD_mat_scaled.dot(s)
most_similar_ply = sim.argmax() # the most similar plylst

display(val.iloc[[idx]])

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
17780,"[러브, 달달, 사랑, 연인]",119302,,"[342791, 408870, 371998, 426500, 549178, 29525...",3,2020-04-14 12:07:51.000,"[194, 626, 61, 846, 804, 804, 266, 145, 642, 8...","[1084, 1046, 1063]","[194, 626, 61, 846, 804, 804, 266, 145, 642, 8..."


In [ ]:
# 위에서 추출한 가장 유사한 plylst 의 실제 값 확인해보기
tr.query(f'id=={train_obj.to_raw_uid(most_similar_ply)}')

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
26029,"[감성, 매장음악, 추억, 드라이브, 휴식, 기분전환, 일상, 인기, 사랑, 신나는]",99168,하트수 10만 이상 히트곡 리스트❤️ (많은순 정렬),"[387859, 654757, 497066, 13198, 454218, 610933...",56,2020-04-22 18:49:20.000,"[79, 804, 804, 510, 194, 145, 145, 626, 626, 7...",[],"[79, 804, 804, 510, 194, 145, 145, 626, 626, 7..."


In [ ]:
#########
# songs #
#########

song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

songs = set(target.songs)
len_ = len(songs)

n = 100
end_num = len_ + n

# 점수를 고려하여 cluster별로 노래를 선택합니다.
# ex : cluster 1에 대한 점수가 0.3인 경우 : cluster 1에서 30개의 노래를 빈도 순으로 추출한다.

for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
    cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
    display(song_meta.query(f'new_song_id == {cluster_id}').head())
    num = int(n*ratio) + 1
    limit =len_ + num

    for song in song_freq_dict.loc[str(cluster_id)]:
        end_state = (len(songs) >= end_num)
        if (len(songs) >= limit) or end_state:
            break
        songs.add(str(song))
    
    if end_state:
        break
    len_ = len(songs)

recommended_songs = songs - set(target.songs)

print("Ground truth :", [sid_to_song[i] for i in target.songs])
print("Recommended songs :")
for i in recommended_songs:
    print(sid_to_song[int(i)])

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
311,"[GN0401, GN0403, GN0402]",20151012,No Make Up,2644221,[537920],No Make Up,[GN0400],[Zion.T],311,"[휴식, 봄, 카페, 달달한, 여유, 오후, 매장, 힐링, 기분전환, 일상, 매장음...",857
2230,"[GN0401, GN0402]",20181010,삐삐,10210535,[261143],삐삐,[GN0400],[아이유],2230,"[댄스, 저녁, 카페, 여름밤, 매장, 드라이브, 퇴근길, 여름, 감성, 기분전환,...",857
4984,"[GN1301, GN1308, GN1302]",20190614,Volume 1 EP,10296836,[2735431],Honesty,[GN1300],[Pink Sweat$],4984,"[비오는날, 감성, Pop, 그루브, 위로, 기분전환, 팝송, 팝, 가을, 휴식, ...",857
5844,"[GN0401, GN0403, GN1501, GN1504]",20190509,A-TEEN2 Part.1,10282644,[698776],스며들기 좋은 오늘,"[GN0400, GN1500]",[백예린],5844,"[위로, 힐링, 감성, 여름밤, 설렘, 사랑, 비오는날, 저녁, 카페, 주말, 오후...",857
7070,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20170613,남이 될 수 있을까,10070846,"[792022, 714975]",남이 될 수 있을까,"[GN0500, GN0800]","[볼빨간사춘기, 스무살]",7070,"[카페, 주말, 달달한, 발라드, 드라이브, 봄, 힐링, 휴식, 잔잔한, 감성, 여...",857


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
250,"[GN0105, GN0101]",20130404,The 3rd Album Part.2 `Love Blossom (러브블러썸)`,2180379,[175139],Love Blossom (러브블러썸),[GN0100],[케이윌],250,"[봄, 힐링, 사랑, 신나는, 휴식, 잔잔한, 까페, 설렘, 운동, 달달한, 매장,...",804
5871,"[GN0105, GN0101]",20181226,너는 어떻게 (Feat. 백예린),10236698,[1956062],너는 어떻게 (Feat. 백예린),[GN0100],[구원찬],5871,"[봄, 따뜻한, 사랑, 겨울, 감성, 밤, 알앤비, 새벽감성, 새벽, 힐링, 잠들기...",804
6546,"[GN0105, GN1501, GN0101, GN1504]",20140902,연애의 발견 OST Part 4,2279896,[192827],"묘해, 너와","[GN1500, GN0100]",[어쿠스틱 콜라보],6546,"[추억, 회상, 겨울, 휴식, 매장음악, 어쿠스틱, 카페, 슬픔, 가을, 사랑, 기...",804
6716,"[GN0105, GN1501, GN0101, GN1504]",20161225,도깨비 OST Part.6,10025804,[757273],Who Are You,"[GN1500, GN0100]",[샘김 (Sam Kim)],6716,"[혼자, 새벽감성, 비오는날, 감성, Pop, 그루브, 위로, 기분전환, 팝송, 팝...",804
8719,"[GN0105, GN0101]",20151023,CHAT-SHIRE,2646282,[261143],무릎,[GN0100],[아이유],8719,"[슬픔, 추억, 이별, 회상, 감성, 잔잔한, 밤, 새벽, 발라드, 알앤비, 비오는...",804


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
642,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20171030,twicetagram,10107114,[905701],LIKEY,"[GN2500, GN0200]",[TWICE (트와이스)],642,"[댄스, 힙합, 기분좋은, 운동, 드라이브, 스트레스, 오후, 여름, 시원한, 여행...",194
5385,"[GN0303, GN0301]",20180526,3곡,10170534,[585492],빈집 (Prod. By dnss) (Feat. THAMA),[GN0300],[기리보이],5385,"[감성힙합, 힙합, 알앤비, 휴식, 가을, 카페, 감성, 사랑, 이별, 랩, 위로,...",194
7749,"[GN0303, GN0301]",20071104,Lonely Night,360178,"[428847, 428848]",Lonely Night,[GN0300],"[사이먼 도미닉, 8℃ Boyz]",7749,"[힐링, 휴식, 잔잔한, 스트레스, 기분전환, 감성힙합, 느낌있는, 힙합, 랩, 락...",194
10427,"[GN0303, GN0301]",20151231,또 하루,2659810,[1732],또 하루 (Feat. 개코),[GN0300],[개리],10427,"[기분전환, 휴식, 느낌있는, 주말, 힐링, 밤, 새벽, 그리움, 드라이브, 잔잔한...",194
11093,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20190412,MAP OF THE SOUL : PERSONA,10273641,[672375],작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),"[GN2500, GN0200]",[방탄소년단],11093,"[여름, 드라이브, 아이돌, 발라드, 감성, Pop, HipHop, 휴식, 기분전환...",194


Ground truth : ['I Love U Oh Thank U (Feat. 김태우 Of God)', '그런사람이기를', 'Special Girl (Feat. Bumkey)', 'Bon Voyage (Feat. Beenzino)', '금요일에 만나요 (Feat. 장이정 Of HISTORY)', '너의 모든 순간', '완벽한 너 (Feat. 제이디)', '무지개 (Feat. 윤닭 Of 오브로젝트)', '니꺼내꺼', 'Think About` Chu (Prod. By 박근태)', '썸 탈꺼야', '지금 뭐해 (Crush on you)', 'It`s you', '사랑이라 하자 (Prod. 로코베리)', 'It`s You (Feat. ZICO)', '스며들기 좋은 오늘', 'When I fall in love (Feat. Meego, 수란)', 'Nerdy Love (Feat. 백예린)', '축가 (Marry Me)', '베개']
Recommended songs :
코끝에 겨울
Rain Drop
지금껏 그랬듯 앞으로도 계속
Call You Mine (Feat. Geologic Of The Blue Scholars)
Love Yourself
Lost Stars
끝사랑
그냥 (Just)
첫 이별 그날 밤
우산 (Feat. 윤하)
오늘부터 우리는 (Me Gustas Tu)
봄 사랑 벚꽃 말고
잊어버리지마 (Feat. 태연)
Shape of You
너의 모든 순간
듣는편지
광화문에서 (At Gwanghwamun)
듣고 있어?
비가 오는 날엔
시간을 달려서 (Rough)
Groovin`
죽일 놈 (Guilty)
Love Again
걱정말아요 그대
감아 (Feat. Crush)
나만 몰랐던 이야기
한여름밤의 꿀
양화대교
Ice Cream Cake
내가 말하고 싶은건
YOUTH
Thinking Out Loud
사랑이란 멜로는 없어
안아줘
Sugar
Oasis (Feat. ZICO)
Bye bye my blue
묘해, 너와
이 소설의 끝을 다시 써보려 해
빨간 맛 (Red Flav

In [ ]:
########
# tags #
########

tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

recommended_tags = set(target.tags)
len_ = len(recommended_tags)

for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
    recommended_tags.add(train_obj.to_raw_iid(tag_id))
    if len(recommended_tags) == 10 + len_ :
        break

recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

print("Ground truth :", target.tags)
print("Recommended tags :",recommended_tags)

Ground truth : ['러브', '달달', '사랑', '연인']
Recommended tags : ['여행', '아침', '퇴근길', '주말', '클럽', '느낌있는', '랩', '달달', '설렘', '공부']


# Prediction_example 2 : 카페, 독서, 뉴에이지


In [ ]:
############# 1. SVD 행렬을 행 별로 normalize 시킨다.

n_values = n_clusters + n_tags
org = user_matrix.dot(item_matrix.T) # org_matrix
norm = np.linalg.norm(org,axis=1).reshape(-1,1)
SVD_mat_scaled = org/norm

In [239]:
idx = 1046 # 배경음악, 카페, 독서, 뉴에이지 등
target = val.iloc[idx]

# extract clusters/tags 
values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

# convert to one-hot vector
s = np.eye(n_values)[values].sum(axis=0)

# calcuate similarity
sim = SVD_mat_scaled.dot(s)
most_similar_ply = sim.argmax() # the most similar plylst

display(val.iloc[[idx]])

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
1046,"[배경음악, 카페, 독서, 뉴에이지, 아침]",102119,,"[439336, 335116, 552555, 17328, 29793, 113724,...",2,2020-01-13 14:24:31.000,"[70, 133, 302, 113, 133, 52, 302, 855, 302, 76...","[1033, 1042, 1017, 1048]","[70, 133, 302, 113, 133, 52, 302, 855, 302, 76..."


In [240]:
# 위에서 추출한 가장 유사한 plylst 의 실제 값 확인해보기
tr.query(f'id=={train_obj.to_raw_uid(most_similar_ply)}')

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
60166,"[카페, 연주곡, 매장, bgm, 독서, 뉴에이지, 가을, 공부, 밤]",137903,가을날 카페에서 틀어주는 피아노 BGM,"[304995, 131690, 645171, 622433, 251746, 64691...",12,2019-01-29 19:13:13.000,"[214, 214, 214, 457, 855, 821, 214, 821, 302, ...",[],"[214, 214, 214, 457, 855, 821, 214, 821, 302, ..."


In [241]:
#########
# songs #
#########

song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

songs = set(target.songs)
len_ = len(songs)

n = 100
end_num = len_ + n

# 점수를 고려하여 cluster별로 노래를 선택합니다.
# ex : cluster 1에 대한 점수가 0.3인 경우 : cluster 1에서 30개의 노래를 빈도 순으로 추출한다.

for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
    cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
    display(song_meta.query(f'new_song_id == {cluster_id}').head())
    num = int(n*ratio) + 1
    limit =len_ + num

    for song in song_freq_dict.loc[str(cluster_id)]:
        end_state = (len(songs) >= end_num)
        if (len(songs) >= limit) or end_state:
            break
        songs.add(str(song))
    
    if end_state:
        break
    len_ = len(songs)

recommended_songs = songs - set(target.songs)

print("Ground truth :", [sid_to_song[i] for i in target.songs])
print("Recommended songs :")
for i in recommended_songs:
    print(sid_to_song[int(i)])

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
580,"[GN1806, GN1801]",20161128,겨울밤,10018484,[944046],겨울밤,[GN1800],[프링 (Pring)],580,"[카페, 잔잔한, 커피, 기분전환, 피아노, 새벽, 뉴에이지, 자장가, 밤, 아침,...",821
3603,[GN1801],20170120,밤노을,10032740,[713816],밤노을,[GN1800],[루바토],3603,"[피아노, 연주곡, 집중, 독서, 공부, 감성, 주말, 잔잔한, 오후, 휴식, 가을...",821
4034,"[GN1806, GN1801]",20150414,Happy Spring Day,2313730,[713917],우리 동네에 햇살이 내리면,[GN1800],[러브래빗],4034,"[여유, 연주곡, 피아노, 독서, 카페, 자장가, 집중, 잠들기전, 공부, 뉴에이지...",821
5724,[GN1801],20121224,The Place Dancing With Snow,2170846,[695258],찻잔과 도넛이 춤추는 가게 (Christmas Ver.),[GN1800],[세레노],5724,"[휴식, 밤, 잔잔한, 새벽, 감성, 뉴에이지, 피아노, 새벽감성, 겨울, 까페, ...",821
6868,"[GN1806, GN1801]",20160516,너의 색으로 물들다,2684910,[476012],너의 색으로 물들다,[GN1800],[신지호],6868,"[뉴에이지, 집중, 자장가, 공부, 가을, 피아노, 커피, 독서, 비오는날, 힐링,...",821


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
579,"[GN1806, GN1801]",20170516,f l o w e r,10062379,[8048],f l o w e r,[GN1800],[이루마],579,"[겨울, 휴식, 아침, 연주곡, 오후, 주말, 위로, 뉴에이지, 가을, 잔잔한, 피...",302
1867,"[GN1806, GN1801]",20150210,우리 동네에 햇살이 내리면,2303885,[713917],우리 동네에 햇살이 내리면,[GN1800],[러브래빗],1867,"[스트레스, 연주곡, 여름, 잔잔한, 피아노, OST, 뉴에이지, 비오는날, 감성,...",302
6945,"[GN1806, GN1801]",20161031,위로가 필요한 밤에,10010784,[1227992],위로가 필요한 밤에,[GN1800],[Little Fish],6945,"[감성, 매장음악, 연주곡, 가을, 새벽, 피아노, 커피, 뉴에이지, 위로, 힐링,...",302
10532,"[GN1805, GN1803, GN1801]",20060419,Ciao! Bravo!!,325044,[172765],Kitto Mata Itsuka (분명 다시 언젠가),[GN1800],[Depapepe],10532,"[연주곡, 카페, 피아노, 힐링, 기분전환, 뉴에이지, 스트레스, 휴식, 노동요, ...",302
12939,"[GN1503, GN1501]",19940000,러브 어페어 OST,27359,[2988],Piano Solo,[GN1500],[Ennio Morricone],12939,"[연주곡, 카페, 피아노, 힐링, 기분전환, 뉴에이지, 스트레스, 휴식, 잔잔한, ...",302


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
820,"[GN1802, GN1801]",20110706,영화속 주옥 같은 OST 테마음악,1343461,[554320],Romance (금지된장난),[GN1800],[Trevor Nasser],820,"[카페, 잔잔한, 오후, 피아노, 뉴에이지, 잠들기전, 자장가, 감성, 연주곡, 휴...",214
950,[GN1801],20181222,"명상, 요가, 숙면, 불면증, 우울증 치료에 좋은 뮤직테라피 뉴에이지 피아노 ASM...",10235953,[2203026],별과 별 사이에 (잔잔한 빗소리),[GN1800],[우리동네피아노],950,"[연주곡, 잔잔한, 집중, 공부, 밤, 피아노, 뉴에이지, 힐링, 새벽, 휴식, 매...",214
1518,"[GN1806, GN1801]",20160930,피아노,10002690,[721438],새벽 편지,[GN1800],[피아노],1518,"[휴식, 혼자, 잔잔한, 위로, 힐링, 뉴에이지, 감성, 아침, 출근길, 기분전환,...",214
2434,[GN1801],20180112,La Granada,10127814,[677652],La Granada (Piano Ver.),[GN1800],[최병욱],2434,"[힐링, 뉴에이지, 주말, 연주곡, 스트레스, 기분전환, 집중, 위로, 잠들기전, ...",214
2931,[GN1801],20171206,힐링을 위한 감성 피아노 연주곡 베스트,10117047,[2019643],"바다, 그리고 기억 (Sea Ver.)",[GN1800],[명상음악],2931,"[피아노, 힐링, 집중, 노동요, 카페, 연주곡, 공부, 휴식, 위로, 감성, 자장...",214


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
110,"[GN1403, GN1401]",20180907,20th Century Masters: The Millennium Collectio...,5643752,[30262],Wayfaring Pilgrim,[GN1400],[Roy Buchanan],110,"[록, 기분전환, Rock, 락, 신나는]",362
291,"[GN1402, GN1401]",20010424,Natural Woman The Very Best Of Carole King,907,[12039],Up On The Roof (Live),[GN1400],[Carole King],291,[락],362
620,"[GN1403, GN1401]",20050927,Bring `Em In,310690,[27343],Cheaper To Keep Her/Blues In The Night,[GN1400],[Buddy Guy],620,"[감성, 락]",362
1028,"[GN1402, GN1401, GN1405]",19710000,Blue,26802,[26645],Little Green,[GN1400],[Joni Mitchell],1028,"[감성, 기분전환, 밤, 락]",362
1030,"[GN1402, GN1401]",20010821,"Parsley, Sage, Rosemary And Thyme (Bonus Track)",338140,[102837],The 59Th Street Bridge Song (Feelin` Groovy),[GN1400],[Simon & Garfunkel],1030,[락],362


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
683,[GN1801],20190319,The End Of The World,10262305,[2635414],When You Feel Love,[GN1800],[TSUUDEN YOSHIZO],683,"[출근길, 아침, 새벽, 위로, 잠들기전, 퇴근길, 밤, 휴식, 연주곡, 오후, 피...",190
1552,"[GN2806, GN2801]",20190315,사랑스러운 반려동물을 위한 명품 피아노 모음집,10261184,[2634990],나비야 (바닷가 Ver.),[GN2800],[강아지 음악],1552,"[집중, 연주곡, 피아노, 기분전환, 공부, 노동요, 오후, 독서, 뉴에이지, 잠들...",190
2996,[GN1801],20180122,포근하고 편안한 숙면을 위한 잔잔하고 따뜻한 감성 피아노 연주곡 베스트,10130962,[2062502],달콤한 잠 (Sweet Dream),[GN1800],[자장가],2996,"[연주곡, 피아노, 자장가, 편안한, 집중, 오후, 뉴에이지, 잠들기전, 휴식, 노...",190
3768,[GN1801],20180102,마음의 평온을 위한 감성 명상 힐링 피아노 연주곡 베스트,10124107,[2059910],우주의 소리 (aum),[GN1800],[요가음악],3768,"[아침, 스트레스, 자장가, 운동, 힐링, 뉴에이지, 노동요, 피아노, 공부, 집중...",190
5377,"[GN1701, GN1703]",20150630,로맨틱 샘물고기,2326625,[873263],이상한 꿈 (String Ensembles With Piano),[GN1700],[이샘물],5377,"[여유, 힐링, 편안한, 뉴에이지, 봄, 카페, 휴식, 연주곡, 피아노, 잠들기전,...",190


Ground truth : ['Danny Boy', 'Merry Christmas Mr. Lawrence', 'Reminiscent (회상)', '그럴 수 있다면', 'River Flows In You', '다시 돌아온 계절', 'Chaconne', '함께 걷던 길', 'Suddenly', '낯선 하루 (A Strange Day)', '오늘은 왠지...', '이제는 쉬어도 돼', '아픔을 잊는 법', '그대를 생각했던 순간들', '내리던 가을잎은', '그 집', '환상(illusion fantasy)', '위로가 필요한 밤', '놀이터', '기도', '마음정원']
Recommended songs :
Blue Sea
반갑다 봄아
사랑하는 이들에게
명상음악 (조용한 사색)
쉬고 싶은 나무
Riding With The King
Moonrise
Angel Eyes
leaf
고양이의 하루
마음정원
울면 안돼
May Be
봄의 왈츠
Wayfaring Stranger
여행
비가 그치고 난뒤(after the rain)
위로
Smile Smile Smile
지금은 우리가 멀리 있을지라도
Chopin : Waltz No.9 In A Flat Major Op.69 `L`Adieu` (쇼팽 : 왈츠 9번 내림 가장조 작품번호 69 `이별의 왈츠`)
Spring Breeze Waltz
창
This Is The Life
Day Is Done
Suddenly
배낭여행
언제나 몇 번이라도 (센과 치히로의 행방불명)
뉴에이지 힐링의 시간, 추억
Happy And Sad
녹턴
부탁해요
이제는 안녕
Happy Birthday
석모도 가는 길
Hallelujah
그대를 생각했던 순간들
사랑의 인사
Lie To Me
가장 조용했던 밤의 위로
지난 날 꿈처럼
따뜻한 그날
토닥토닥 괜찮아
Daily Life
별의 바다 (Sea Of Stars)
자장가
Iihidattane (좋은날이었어)
저 별 아래 어딘가에
Madam Windham Hill
환상(illusion fantasy)
Little Flo

In [54]:
########
# tags #
########

tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

recommended_tags = set(target.tags)
len_ = len(recommended_tags)

for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
    recommended_tags.add(train_obj.to_raw_iid(tag_id))
    if len(recommended_tags) == 10 + len_ :
        break

recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

print("Ground truth :", target.tags)
print("Recommended tags :",recommended_tags)

Ground truth : ['배경음악', '카페', '독서', '뉴에이지', '아침']
Recommended tags : ['독서', '혼자', '편안한', '스트레스', '집중', '연인', '연주곡', '따뜻한', '매장', '저녁']


# Prediction example 3 : 비오는날, 발라드, 우울

In [259]:
idx = 5982
target = val.iloc[idx]

# extract clusters/tags 
values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

# convert to one-hot vector
s = np.eye(n_values)[values].sum(axis=0)

# calcuate similarity
sim = SVD_mat_scaled.dot(s)
most_similar_ply = sim.argmax() # the most similar plylst

display(val.iloc[[idx]])

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
5982,"[비오는날, 어쿠스틱, 발라드, 우울, 분위기]",69439,,"[208812, 232874, 389468, 95323, 613722, 557120...",1,2019-12-20 06:27:00.000,"[365, 804, 890, 133, 857, 133, 857, 56, 857, 4...","[1000, 1065, 1020, 1054, 1037]","[365, 804, 890, 133, 857, 133, 857, 56, 857, 4..."


In [260]:
# 위에서 추출한 가장 유사한 plylst 의 실제 값 확인해보기
tr.query(f'id=={train_obj.to_raw_uid(most_similar_ply)}')

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
8624,"[외로움, 저녁, 밤, 회상, 추억, 슬픔, 이별, 쓸쓸함, 감미로운, 새벽]",61937,달빛아래 들리는 감미로운 목소리.,"[208812, 629738, 232874, 13281, 389468, 335416...",1,2020-03-17 06:52:06.000,"[365, 857, 804, 145, 890, 485, 857, 133, 56, 8...",[],"[365, 857, 804, 145, 890, 485, 857, 133, 56, 8..."


In [261]:
#########
# songs #
#########

song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

songs = set(target.songs)
len_ = len(songs)

n = 100
end_num = len_ + n

# 점수를 고려하여 cluster별로 노래를 선택합니다.
# ex : cluster 1에 대한 점수가 0.3인 경우 : cluster 1에서 30개의 노래를 빈도 순으로 추출한다.

for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
    cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
    display(song_meta.query(f'new_song_id == {cluster_id}').head())
    num = int(n*ratio)*1.2 + 1
    limit =len_ + num

    for song in song_freq_dict.loc[str(cluster_id)]:
        end_state = (len(songs) >= end_num)
        if (len(songs) >= limit) or end_state:
            break
        songs.add(str(song))
    
    if end_state:
        break
    len_ = len(songs)

recommended_songs = songs - set(target.songs)

print("Ground truth :", [sid_to_song[i] for i in target.songs])
print("Recommended songs :")
for i in recommended_songs:
    print(sid_to_song[int(i)])

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
54,[GN0101],20190408,있어줄래,10271322,[500421],있어줄래,[GN0100],[길구봉구],54,"[여름, 휴식, 기분전환, 봄, 사랑, 발라드, 감성, 드라이브, 카페, 여유, 커...",485
496,"[GN0105, GN0101]",20190923,나만 아픈 일,10331095,[259769],나만 아픈 일,[GN0100],[이해리 (다비치)],496,"[잔잔한, 밤, 새벽, 발라드, 알앤비, 추억, 슬픔, 저녁, 사랑, 이별, 회상,...",485
1513,"[GN0105, GN0101]",20180313,듣고 있니,10146980,[1956715],듣고 있니,[GN0100],[러브 포엠],1513,"[겨울, 휴식, 발라드, 잔잔한, 드라이브, 산책, 이별, 사랑, 인디, 감성, 혼...",485
15318,"[GN0105, GN0101]",20171218,With Winter,10120289,[726168],With Winter,[GN0100],[민은서밴드],15318,"[어쿠스틱, 혼자, 발라드, 우울, 분위기, 드라이브, 산책, 인디, 저녁, 밤, ...",485
15910,"[GN0105, GN1501, GN0101, GN1504]",20180529,기름진 멜로 OST Part.4,10171432,[511108],매일매일 그리울거야,"[GN1500, GN0100]",[주영],15910,"[감성, 가을, 카페, 발라드, 오후, 매장, 드라이브, 저녁, 이별, 밤, 여름밤...",485


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
14669,"[GN0104, GN0101]",20080228,창문을 열어놓고 (All Star 2집 Vol.2),368897,"[27033, 58979]",창문을 열어놓고,[GN0100],"[별, 나윤권]",14669,"[까페, 설렘, 사랑, 발라드, 추억, 매장, 드라이브, 감성, 기분전환, 달달한,...",273
14847,"[GN0501, GN0101, GN0506, GN0509]",20180130,I Miss You,10134513,[763813],I Miss You,"[GN0500, GN0100]",[로코베리],14847,"[휴식, 감성, 가을, 카페, 새벽, 드라이브, 여행, 위로, 기분전환, 사랑, 이...",273
15211,"[GN0105, GN0101]",20141216,달에게,2295258,[782798],달에게,[GN0100],[피그말리온],15211,"[여행, 기분전환, 산책, 겨울, 까페, 감성, 카페, 잔잔한, 명곡, 사랑, 띵곡...",273
17648,"[GN1501, GN0801, GN1504]",20170809,다시 만난 세계 OST Part.4,10086459,[756963],다시 만날거야,"[GN1500, GN0800]",[윤딴딴],17648,"[매장음악, 봄, 카페, 드라이브, 재즈, 여행, 사랑, 기분전환, 어쿠스틱, 인디...",273
17930,"[GN0105, GN0101]",20141229,High Heel Project #1. Parting,2296997,[826743],그곳엔 (Vocal 제이오) With 소란 By Guitar 이태욱,[GN0100],[하이힐],17930,"[힐링, 휴식, 잔잔한, 추억, 기분전환, 회상, 드라이브, 발라드, 까페, 취향저...",273


Ground truth : ['사랑하는 법을 몰라서', 'Rain Drop', '이별을 건너다', '내일 할 일', 'Rain', '가끔 내가', '같은 시간 속의 너', 'Rainy', '어떻게 지내', '울컥해', '한번쯤 (With 박은옥)', '후회 (Feat. 박은옥)', '이별의 이유 (Feat. 박은옥)', '연인', '너로 가득해', '벙어리', 'MOM (겨울나무)', '사랑을 놓치고', 'ALONE (Vocal 김제훈)', '미련이 남아서', '사랑이 그리워', '은퇴식', '손편지 (Vocal 태인)', '비가 내렸어 (Vocal by 스티브언니)', '지워줄게 (Vocal by 스티브언니)', '이 밤', '우산을 쓰고', '머무는 별', '안아줘', 'Sad Movie (Vocal by Levi)', '지금보다 조금 (Feat. 이원)', '이 노래를 듣게 된다면 (Feat. 이소진)', '꽃잎이 떨어지면 (Feat. 뜨때로)']
Recommended songs :
사랑을 놓치고
니가 없다
듣지 못할 너에게 (Prod. 김준수)
다음이 있을까
내 목소리 들리니
우리
내가 사라졌으면 좋겠어
어때
있어줘요
미움 (Vocal 손주희)
지워줄게 (Vocal by 스티브언니)
너로 가득해
Midnight (Feat. 민우)
미지
오늘 뭐 했는지 말해봐
안녕 내사랑
축가 (Marry Me)
그때 우린
선물
나이값을 못해
헤어진 우리가 지켜야 할 것들
사랑이 우릴 다시 만나게 한다면
이 거리의 끝이 오면
언능언능
Love Road
동화
함께 걷던 길 (Duet With. April)
우리 왜 헤어져야 해
헤어지는 길 (with 신성진)
내게 다시오면
Sad Movie (Vocal by Levi)
벙어리
따뜻해
이별의 이유 (Feat. 박은옥)
미련이 남아서
빈 방
연인
잊는다는 게
지울수 없는 지우개
그 겨울
불안해
내 곁에 있어줘
함께 걷던 길 (With 선진)
이별오후
듣고 있니
꽃 처럼 한 철만 사랑해 줄 건가요?
사랑 둘
분홍빛 가득한 날에 

In [262]:
########
# tags #
########

tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

recommended_tags = set(target.tags)
len_ = len(recommended_tags)

for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
    recommended_tags.add(train_obj.to_raw_iid(tag_id))
    if len(recommended_tags) == 10 + len_ :
        break

recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

print("Ground truth :", target.tags)
print("Recommended tags :",recommended_tags)

Ground truth : ['비오는날', '어쿠스틱', '발라드', '우울', '분위기']
Recommended tags : ['달달한', '위로', '여름밤', '휴식', '설렘', '우울', '취향저격', '소울', '산책', '노래방']


# Prediction example 4 : 힙합, 드라이브, 센치

In [251]:
idx = 11596
target = val.iloc[idx]

# extract clusters/tags 
values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

# convert to one-hot vector
s = np.eye(n_values)[values].sum(axis=0)

# calcuate similarity
sim = SVD_mat_scaled.dot(s)
most_similar_ply = sim.argmax() # the most similar plylst

display(val.iloc[[idx]])

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
11596,"[힙합, 카페, 드라이브, 센치, 커피]",18426,,"[251980, 702912, 133859, 372171, 240842, 27814...",1,2020-02-24 00:29:18.000,"[266, 266, 266, 194, 266, 194, 194, 289, 266, ...","[1030, 1033, 1001, 1097, 1096]","[266, 266, 266, 194, 266, 194, 194, 289, 266, ..."


In [252]:
# 위에서 추출한 가장 유사한 plylst 의 실제 값 확인해보기
tr.query(f'id=={train_obj.to_raw_uid(most_similar_ply)}')

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
20299,"[힙합, 느낌있는, 카페, 그루비, 매장, 센치, 기분전환, 산책, 트렌디, 알앤비]",17684,센치한 주말 트렌디한 노래가 들려오는 카페,"[331975, 251980, 702912, 133859, 372171, 24084...",3,2020-03-30 08:58:53.000,"[194, 266, 266, 266, 194, 266, 194, 857, 194, ...",[],"[194, 266, 266, 266, 194, 266, 194, 857, 194, ..."


In [253]:
#########
# songs #
#########

song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

songs = set(target.songs)
len_ = len(songs)

n = 100
end_num = len_ + n

# 점수를 고려하여 cluster별로 노래를 선택합니다.
# ex : cluster 1에 대한 점수가 0.3인 경우 : cluster 1에서 30개의 노래를 빈도 순으로 추출한다.

for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
    cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
    display(song_meta.query(f'new_song_id == {cluster_id}').head())
    num = int(n*ratio) + 1
    limit =len_ + num

    for song in song_freq_dict.loc[str(cluster_id)]:
        end_state = (len(songs) >= end_num)
        if (len(songs) >= limit) or end_state:
            break
        songs.add(str(song))
    
    if end_state:
        break
    len_ = len(songs)

recommended_songs = songs - set(target.songs)

print("Ground truth :", [sid_to_song[i] for i in target.songs])
print("Recommended songs :")
for i in recommended_songs:
    print(sid_to_song[int(i)])

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
3312,"[GN0509, GN0501, GN0304, GN0505, GN0301]",20150206,Puff Puff Pass It,2303447,[618913],Puff Puff Pass It (Feat. Life of Hojj),"[GN0500, GN0300]",[도넛맨 (Donutman)],3312,"[힙합, HipHop, 노래, 잔잔한, 밤, 새벽, 랩, 힐링, 휴식, 기분전환, ...",266
3554,"[GN0501, GN0304, GN0505, GN0301]",20110127,파티가 끝난 후,1156451,"[227557, 326266]",파티가 끝난 후 (Feat. 엄지희),"[GN0500, GN0300]","[제이에이 (JA), 자이언]",3554,"[감성, 잔잔한, 추억, 위로, 휴식, 힐링, 분위기, 새벽, 힙합, 알앤비, 드라...",266
6701,"[GN0303, GN0301]",20160218,만화처럼,2667864,"[219621, 446175]",만화처럼 (Like Romance Comics),[GN0300],"[매드클라운, 브라더수]",6701,"[오후, 힙합, 취향저격, 감각적인, 드라이브, 그루브, 여행, 기분전환, 트렌디,...",266
8922,"[GN0501, GN0304, GN0505, GN0301]",20180621,SSANGMOON,10177626,[896123],쌍문 (SSANGMOON) (Feat. 이하츠),"[GN0500, GN0300]",[STEN (스텐)],8922,"[힙합, 매장음악, 카페, 알앤비, 감각적인, 그루브, 커피, 트렌디, 휴식, 운동...",266
10179,"[GN0302, GN0301]",20181020,쇼미더머니 777 Episode 3,10214285,[957315],Save (Feat. 팔로알토)(Prod. 코드 쿤스트),[GN0300],[루피 (Loopy)],10179,"[힙합, 랩, 기분전환, 감성힙합, 느낌있는, 잔잔한, 감성, Pop, HipHop...",266


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
2780,"[GN0302, GN0301]",20190928,쇼미더머니 8 Final,10333372,[782842],미안해서 미안해 (Feat. 권정열 of 10cm),[GN0300],[펀치넬로 (punchnello)],2780,"[랩, 밤, 감각적인, 분위기, 드라이브, 기분전환, 퇴근길, 저녁, 매장음악, 카...",416
5241,"[GN0303, GN0301]",20191004,사인히어(SignHere) episode2,10335427,"[672625, 957445, 672917, 883201]",살아있네 (Prod. GRAY),[GN0300],"[GRAY (그레이), 오사마리, 니화 (NiiHWA), 블랙나인 (BLACK NI...",5241,"[힙합, 트렌디, 드라이브, 기분전환, 팝, 신나는, 비오는날, 감성힙합, 알앤비,...",416
7303,"[GN0302, GN0301]",20200126,BOY,10379635,[873162],BOY,[GN0300],[Kid Milli],7303,"[힙합, 랩, 봄, 카페, 취향저격, 여행, 감성, 기분전환, 신나는, 휴식, 드라...",416
8844,"[GN0303, GN0301]",20170508,예고편,10060041,[768562],"In my car (Feat. 챈슬러, TakeOne)",[GN0300],[프라임보이 (Primeboi)],8844,"[힙합, 알앤비, 휴식, 가을, 카페, 드라이브, 기분전환, 사랑, 감각적인, 분위...",416
15546,"[GN0302, GN0301]",20191229,Cumpilation,10369676,"[2635189, 727519, 907416, 916309]",오갓!,[GN0300],"[Lil tachi, 블랙넛 (Black Nut), 양홍원, YUNHWAY]",15546,"[밤, 감각적인, 분위기, 드라이브, 기분전환, 퇴근길, 느낌있는, 카페, 취향저격...",416


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
642,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20171030,twicetagram,10107114,[905701],LIKEY,"[GN2500, GN0200]",[TWICE (트와이스)],642,"[댄스, 힙합, 기분좋은, 운동, 드라이브, 스트레스, 오후, 여름, 시원한, 여행...",194
5385,"[GN0303, GN0301]",20180526,3곡,10170534,[585492],빈집 (Prod. By dnss) (Feat. THAMA),[GN0300],[기리보이],5385,"[감성힙합, 힙합, 알앤비, 휴식, 가을, 카페, 감성, 사랑, 이별, 랩, 위로,...",194
7749,"[GN0303, GN0301]",20071104,Lonely Night,360178,"[428847, 428848]",Lonely Night,[GN0300],"[사이먼 도미닉, 8℃ Boyz]",7749,"[힐링, 휴식, 잔잔한, 스트레스, 기분전환, 감성힙합, 느낌있는, 힙합, 랩, 락...",194
10427,"[GN0303, GN0301]",20151231,또 하루,2659810,[1732],또 하루 (Feat. 개코),[GN0300],[개리],10427,"[기분전환, 휴식, 느낌있는, 주말, 힐링, 밤, 새벽, 그리움, 드라이브, 잔잔한...",194
11093,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20190412,MAP OF THE SOUL : PERSONA,10273641,[672375],작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),"[GN2500, GN0200]",[방탄소년단],11093,"[여름, 드라이브, 아이돌, 발라드, 감성, Pop, HipHop, 휴식, 기분전환...",194


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
4011,"[GN0303, GN0301]",20150827,The Anecdote,2336976,[651631],Writer`s Block,[GN0300],[E SENS],4011,"[힙합, 랩, 새벽, 위로, 힐링, 감성힙합, 알앤비, 트렌디, 휴식, 여행, 산책...",147
4412,"[GN0303, GN0301]",20150623,MULTILLIONAIRE,2325598,[314367],111%,[GN0300],[Dok2],4412,"[힐링, 휴식, 밤, 새벽, 힙합, 랩, 감성, 감성힙합, 기분전환, 여행, 드라이...",147
6453,[GN0301],20020911,Kandid Collection Vol. 1,23097,[7561],Unspoken Love,[GN0300],[Kricc (크릭)],6453,"[드라이브, 가을, 여행, 힐링, 휴식, 스트레스, 기분전환, 재즈, 잔잔한, 밤,...",147
7149,"[GN0303, GN0301]",20111007,Simon Dominic Presents `SNL LEAGUE BEGINS`,2017406,[428847],끈 (No More) (Feat. Junggigo),[GN0300],[사이먼 도미닉],7149,"[스트레스, 기분전환, 랩, 감성, 힙합, 사랑, 일상, 드라이브, 운동, 발라드,...",147
8187,"[GN0303, GN0301]",20150527,Andre,2320302,[511640],"SOLD OUT (Feat. 타블로, 자이언티, 로꼬)",[GN0300],[얀키 (YANKIE)],8187,"[힙합, 취향저격, 랩, RnB, 감성, 드라이브, 매장음악, 밤, 새벽, 그루브,...",147


Ground truth : ['The Anecdote', '완벽한 너 (Feat. 제이디)', '현대생활백수 (Feat. 순둥이와달봉이)', 'City', '계속하면 돼 (Feat. 둑바 & 주지운)', '남아있어 (Feat. Crush)', '지나쳐 (Feat. DEAN)', '또 (Feat. Tiger JK, Bizzy, 마샬(MRSHLL))', 'Color (Feat. Douner)', 'iffy (Prod. GroovyRoom)', 'Downtown Baby', 'Fall In Love (Feat. SE O (세오))', '데몰리션 맨 (Feat. 김종서)', '얼라', 'DAN (Feat. Smokey)', 'Maybe', 'Fuxx The Vibe (Feat. Reppleh)', 'Welcome', 'Scene Stealer', '울타리', 'Gone', '호불호 (Feat. 기리보이) (Prod. By GRAY)', '발할라 (Valhalla) (Feat. Hash Swan & Yami Tommy)', '다시전화해', 'Time', '불꽃 (Feat. 조현아 Of 어반자카파)', '제설', '센치해 (Feat. Swervy)', '혼잣말', '아파 후기']
Recommended songs :
보통연애 (Feat. 박보람)
가뭄 (Feat. Beenzino)
화 (Feat. 진실 Of Mad Soul Child) (Fire)
DSF (Feat. 식케이)
입장정리 (Random)
A Real Man
완벽한 너 (Feat. 제이디)
불면증 (Feat. Bobby Kim)
참고 살아 (be...)
멘트 (Feat. 개코 Of Dynamic Duo)
Supernova (feat. WHO$)
기름 같은걸 끼얹나 (Feat. 뎁(Deb) & Beenzino)
화창한 봄날에 (Feat. Kuan)
죽일 놈 (Guilty)
깜빡 (Feat. Zion.T & Crucial Star)
각자의 새벽 (Feat. DOK2, Beatbox DG)
傷 (상) (Feat. 서

In [254]:
########
# tags #
########

tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

recommended_tags = set(target.tags)
len_ = len(recommended_tags)

for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
    recommended_tags.add(train_obj.to_raw_iid(tag_id))
    if len(recommended_tags) == 10 + len_ :
        break

recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

print("Ground truth :", target.tags)
print("Recommended tags :",recommended_tags)

Ground truth : ['힙합', '카페', '드라이브', '센치', '커피']
Recommended tags : ['피아노', '눈물', '편안한', '주말', '비', 'Rock', '어쿠스틱', '사랑', '설렘', '공부']


# 최종 결과 저장

In [ ]:
answers = []

In [ ]:
n_values = n_clusters + n_tags
org = user_matrix.dot(item_matrix.T) # org_matrix
norm = np.linalg.norm(org,axis=1).reshape(-1,1)
SVD_mat_scaled = org/norm

In [57]:
for idx in range(len(val)):
    target = val.iloc[idx]

    # extract clusters/tags 
    values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

    # convert to one-hot vector
    s = np.eye(n_values)[values].sum(axis=0)

    # calcuate similarity
    sim = SVD_mat_scaled.dot(s)
    most_similar_ply = sim.argmax() # the most similar plylst

    #########
    # songs #
    #########

    song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
    sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

    songs = set(target.songs)
    len_ = len(songs)

    n = 100
    end_num = len_ + n

    for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
        cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
        num = int(n*ratio) + 1
        limit =len_ + num

        for song in song_freq_dict.loc[str(cluster_id)]:
            end_state = (len(songs) >= end_num)
            if (len(songs) >= limit) or end_state:
                break
            songs.add(str(song))
        
        if end_state:
            break
        len_ = len(songs)

    recommended_songs = [int(i) for i in songs - set(target.songs)]

    ########
    # tags #
    ########

    tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
    sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

    recommended_tags = set(target.tags)
    len_ = len(recommended_tags)

    for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
        recommended_tags.add(train_obj.to_raw_iid(tag_id))
        if len(recommended_tags) == 10 + len_ :
            break

    recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

    answers.append({
    "id": target.id,
    "songs": list(recommended_songs),
    "tags": list(recommended_tags)
    })

In [58]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [59]:
# # tag mapping 안됐을 때

# a = pd.DataFrame(answers)
# a['tags'] = a.tags.map(lambda x: [tid_to_tag[t] for t in x])
# a['tags'] = a.tags.map(lambda x: [int(t) for t in x])
# a['songs'] = a['songs'].map(lambda x: [int(s) for s in x])
# res = list(a.transpose().to_dict().values())

In [60]:
write_json(answers,"/content/drive/My Drive/recommendation sys/results_0804.json")

In [61]:
len(answers) == len(val)

True

In [64]:
val.id.tolist() == [pred['id'] for pred in answers]

True